# AAI-511 Final Project

Paul Parks

## Libraries

pretty-midi is used for midi calculations (key, tempo, etc):
https://github.com/craffel/pretty-midi

In [2]:
# %pip install pretty_midi 

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 30.1 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 kB 5.0 MB/s eta 0:00:00
  Created wheel for pretty_midi: filename=pretty_midi-0.2.10-py3-none-any.whl size=5592292 sha256=ce3bcf6604f46bc4cd8b1c2caa6e85f6c8109abe9a3538aa5c82a3c3b0ef55da
  Stored in directory: /Users/pparks/Library/Caches/pip/wheels/cd/a5/30/7b8b7f58709f5150f67f98fde4b891ebf0be9ef07a8af49f25
Successfully built pretty_midi
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import os
import pretty_midi
import numpy as np
import mido
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from sklearn.metrics import classification_report, confusion_matrix

## Data Collection
Data is collected and provided to you.

**Dataset**

The project will use a dataset consisting of musical scores from various composers. Download the dataset from Kaggle websiteLinks to an external site..

https://www.kaggle.com/datasets/blanderbuss/midi-classic-music/data

The dataset contains the midi files of compositions from well-known classical composers like Bach, Beethoven, Chopin, and Mozart. The dataset should be labeled with the name of the composer for each score. Please only do your prediction only for below composers, therefore you need to select the required composers from the given dataset above.

1. Bach
1. Beethoven
1. Chopin
1. Mozart

## Data Pre-processing
Convert the musical scores into a format suitable for deep learning models. This involves converting the musical scores into MIDI files and applying data augmentation techniques.

In [5]:
def load_midi_files(base_dir, composers):
    midi_files = []
    labels = []

    for composer in composers:
        composer_dir = os.path.join(base_dir, composer)
        for root, _, files in os.walk(composer_dir):
            for file in files:
                if file.endswith('.mid') or file.endswith('.midi'):
                    midi_files.append(os.path.join(root, file))
                    labels.append(composer)
    return midi_files, labels

base_dir = './Dataset/midiclassics/'
composers = ['Bach', 'Beethoven', 'Chopin', 'Mozart']

midi_files, labels = load_midi_files(base_dir, composers)

print('Number of MIDI files:', len(midi_files))
print('Number of labels:', len(labels))

# print files per composer
for composer in composers:
    print(f'{composer}: {labels.count(composer)}')

Number of MIDI files: 1530
Number of labels: 1530
Bach: 925
Beethoven: 212
Chopin: 136
Mozart: 257


## Feature Extraction
Extract features from the MIDI files, such as notes, chords, and tempo, using music analysis tools.


In [33]:
MAX_NOTES = 1000
MAX_DURATIONS = 1000
MAX_CHORDS = 1000
MAX_TEMPOS = 100

def pad_or_truncate(array, max_length):
    array = np.array(array)
    if len(array) > max_length:
        return array[:max_length]
    else:
        return np.pad(array, (0, max_length - len(array)), 'constant')

In [34]:
def extract_notes(midi_data):
    notes = []
    durations = []
    for instrument in midi_data.instruments:
        if not instrument.is_drum:
            for note in instrument.notes:
                notes.append(note.pitch)
                durations.append(note.end - note.start)
    return notes, durations

def extract_chords(midi_data):
    chords = []
    for instrument in midi_data.instruments:
        if not instrument.is_drum:
            for note in instrument.notes:
                # A simplistic way to consider a chord if multiple notes are starting at the same time
                chords.append((note.start, note.pitch))
    chords.sort()  # Sort by start time
    return chords

def extract_tempo(midi_data):
    tempos = midi_data.get_tempo_changes()
    return tempos

def extract_key_signature(midi_data):
    key_signatures = midi_data.key_signature_changes
    if key_signatures:
        # Get the first key signature change as the representative key for simplicity
        key_signature = key_signatures[0].key_number
    else:
        # Default key signature if none is found
        key_signature = 0  # C major or A minor
    return key_signature

def extract_features(midi_file):
    try:
        midi_data = pretty_midi.PrettyMIDI(midi_file)
        notes, durations = extract_notes(midi_data)
        chords = extract_chords(midi_data)
        tempos = extract_tempo(midi_data)
        key_signature = extract_key_signature(midi_data)
        return notes, durations, chords, tempos, key_signature
    except mido.KeySignatureError as e:
        print(f"Error processing {midi_file}: {e}")
        return [], [], [], [], []


X = []
y = []

for i, file in enumerate(midi_files):
    composer = labels[i]
    notes, durations, chords, tempos, key_signature = extract_features(file)
    if notes and durations and chords and tempos:
        notes = pad_or_truncate(notes, MAX_NOTES)
        durations = pad_or_truncate(durations, MAX_DURATIONS)
        chords = pad_or_truncate([pitch for _, pitch in chords], MAX_CHORDS)
        tempos = pad_or_truncate(tempos[1], MAX_TEMPOS) if len(tempos) > 1 else np.zeros(MAX_TEMPOS)
        
        features = np.concatenate([
            notes.flatten(),
            durations.flatten(),
            chords.flatten(),
            tempos.flatten(),
            np.array([key_signature])
        ])
        X.append(features)
        y.append(composers.index(composer))
    else:
        print(f"Skipping {file}")

Error processing ./Dataset/midiclassics/Beethoven/Anhang 14-3.mid: Could not decode key with 3 flats and mode 255
Skipping ./Dataset/midiclassics/Beethoven/Anhang 14-3.mid
Error processing ./Dataset/midiclassics/Mozart/Piano Sonatas/Nueva carpeta/K281 Piano Sonata n03 3mov.mid: Could not decode key with 2 flats and mode 2
Skipping ./Dataset/midiclassics/Mozart/Piano Sonatas/Nueva carpeta/K281 Piano Sonata n03 3mov.mid


In [35]:
print('Number of samples:', len(X))
print('Number of labels:', len(y))
print('Feature vector length:', len(X[0]))
# count unique labels
unique_labels = set(y)
print('Unique labels:', unique_labels)

Number of samples: 1528
Number of labels: 1528
Feature vector length: 3101
Unique labels: {0, 1, 2, 3}


In [36]:
X = np.array(X, dtype=object)
y = np.array(y)

scaler = StandardScaler()
X = scaler.fit_transform(np.vstack(X))

In [39]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

## Model Building
Develop a deep learning model using LSTM and CNN architectures to classify the musical scores according to the composer.


In [45]:
input_shape = (X_train.shape[1], 1)

model = Sequential()
model.add(LSTM(128, input_shape=input_shape, return_sequences=True))
model.add(LSTM(64, return_sequences=False))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(composers), activation='softmax'))
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

Epoch 1/20


2024-07-05 20:23:14.469410: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


39/39 [==============================] - 102s 3s/step - loss: 1.0686 - accuracy: 0.6170 - val_loss: 0.9581 - val_accuracy: 0.6405
Epoch 2/20
25/39 [==================>...........] - ETA: 33s - loss: 1.0513 - accuracy: 0.5987

KeyboardInterrupt: 

## Model Training
Train the deep learning model using the pre-processed and feature-extracted data.


In [ ]:
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=20, batch_size=32)

## Model Evaluation
Evaluate the performance of the deep learning model using accuracy, precision, and recall metrics.


In [ ]:
val_loss, val_accuracy = model.evaluate(X_val, y_val, verbose=0)
print(f'Validation Accuracy: {val_accuracy}')

# Calculate additional metrics
y_pred = model.predict(X_val)
y_pred_classes = np.argmax(y_pred, axis=1)

print(classification_report(y_val, y_pred_classes, target_names=composers))
conf_matrix = confusion_matrix(y_val, y_pred_classes)
print(conf_matrix)

## Model Optimization
Optimize the deep learning model by fine-tuning hyperparameters.
